In [13]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [2]:
# list of dates to read in datasets
list_dates = ["05-19", "05-21", "05-23", "05-25", "05-27", "05-29", "06-02", "06-05", "06-08", "06-10", "06-13"]

In [3]:
# create dictionary 
final_data = {}

In [4]:
# reads in each csv
for i in list_dates:
    data = pd.read_csv(f"Boston Celtics/{i}.csv", index_col=0)
    
    #gathers data and adds to dict
    date = data["Date"].tolist()[0]
    data[date] = data["Score"]
    final_data[date] = data[date]

In [5]:
# convert dict to dataframe
data_df = pd.DataFrame(final_data)

In [10]:
tester = pd.DataFrame.transpose(data_df)
columns = tester.columns.tolist()

In [7]:
#create dict
final_result = {}

In [18]:
tester

,Aaron Nesmith,Al Horford,Daniel Theis,Derrick White,Grant Williams,Jaylen Brown,Jayson Tatum,Juwan Morgan,Luke Kornet,Malik Fitts,Marcus Smart,Nik Stauskas,Payton Pritchard,Robert Williams III,Sam Hauser
05/19/2022,1.25,21.75,4.75,NaN,27.00,40.00,43.25,2.25,4.50,NaN,63.25,7.75,18.50,17.50,NaN
05/21/2022,0.00,48.00,5.25,8.00,20.75,54.75,21.00,1.25,0.00,0.0,30.00,3.50,9.50,NaN,NaN
05/23/2022,10.25,34.25,7.75,39.00,16.25,23.75,51.50,0.00,NaN,0.0,NaN,2.00,26.00,27.25,NaN
05/25/2022,0.00,38.25,NaN,27.25,12.50,36.00,49.00,0.00,0.00,0.0,17.25,0.00,1.25,26.50,NaN
05/27/2022,0.00,20.75,NaN,43.25,6.50,37.50,51.75,NaN,NaN,NaN,25.00,NaN,0.00,18.75,NaN
05/29/2022,NaN,33.50,NaN,14.75,19.50,39.00,54.50,NaN,NaN,NaN,47.75,NaN,0.00,6.75,NaN
06/02/2022,0.00,42.00,5.50,28.25,8.25,45.25,39.25,0.00,NaN,0.0,36.25,0.00,19.50,24.50,0.0
06/05/2022,11.50,14.50,9.25,25.50,11.25,29.50,43.00,NaN,3.50,3.5,11.50,8.75,13.50,10.00,5.0
06/08/2022,1.25,34.50,NaN,14.75,18.25,48.75,49.50,0.00,NaN,0.0,39.25,0.00,4.50,34.50,0.0
06/10/2022,NaN,24.00,NaN,20.75,4.75,33.50,52.75,NaN,NaN,NaN,39.00,NaN,1.75,34.00,NaN


In [16]:
add = {}

#iterate through index
for column in columns:
    
# univariate data prep LSTM
    from numpy import array

    # split sequence
    def split_sequence(sequence, n_steps):
        X, y = list(), list()
        for i in range(len(sequence)):
            # find end of pattern 
            end_ix = i + n_steps
            #check if beyond sequence to break
            if end_ix > len(sequence)-1:
                break
            #gather input and output parts of the pattern
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)
    # sequence 
    rent_rate = list(tester[column].dropna())
    # number of steps
    n_steps = 3
    X, y = split_sequence(rent_rate, n_steps)
    # summarize the data
    for i in range(len(X)):
        print(X[i], y[i])

    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], n_features))

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # fit model
    model.fit(X, y, epochs=200, verbose=0)

    # demonstrate prediction
    x_input = array(tester[column][-3:])
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    add[column] = yhat.tolist()[0][0]

[ 1.25  0.   10.25] 0.0
[ 0.   10.25  0.  ] 0.0
[10.25  0.    0.  ] 0.0
[0. 0. 0.] 11.5
[ 0.   0.  11.5] 1.25
[ 0.   11.5   1.25] 4.25
[21.75 48.   34.25] 38.25
[48.   34.25 38.25] 20.75
[34.25 38.25 20.75] 33.5
[38.25 20.75 33.5 ] 42.0
[20.75 33.5  42.  ] 14.5
[33.5 42.  14.5] 34.5
[42.  14.5 34.5] 24.0
[14.5 34.5 24. ] 21.75
[4.75 5.25 7.75] 5.5
[5.25 7.75 5.5 ] 9.25
[ 8.   39.   27.25] 43.25
[39.   27.25 43.25] 14.75
[27.25 43.25 14.75] 28.25
[43.25 14.75 28.25] 25.5
[14.75 28.25 25.5 ] 14.75
[28.25 25.5  14.75] 20.75
[25.5  14.75 20.75] 8.75
[27.   20.75 16.25] 12.5
[20.75 16.25 12.5 ] 6.5
[16.25 12.5   6.5 ] 19.5
[12.5  6.5 19.5] 8.25
[ 6.5  19.5   8.25] 11.25
[19.5   8.25 11.25] 18.25
[ 8.25 11.25 18.25] 4.75
[11.25 18.25  4.75] 11.25
[40.   54.75 23.75] 36.0
[54.75 23.75 36.  ] 37.5
[23.75 36.   37.5 ] 39.0
[36.  37.5 39. ] 45.25
[37.5  39.   45.25] 29.5
[39.   45.25 29.5 ] 48.75
[45.25 29.5  48.75] 33.5
[29.5  48.75 33.5 ] 34.75
[43.25 21.   51.5 ] 49.0
[21.  51.5 49. ] 51.75
[

In [17]:
add

{'Aaron Nesmith': nan,
 'Al Horford': 42.81393051147461,
 'Daniel Theis': nan,
 'Derrick White': 11.567524909973145,
 'Grant Williams': 22.470426559448242,
 'Jaylen Brown': 56.28550338745117,
 'Jayson Tatum': 43.84681701660156,
 'Juwan Morgan': nan,
 'Luke Kornet': nan,
 'Malik Fitts': nan,
 'Marcus Smart': 38.99502182006836,
 'Nik Stauskas': nan,
 'Payton Pritchard': 29.9066162109375,
 'Robert Williams III': 5.160600662231445,
 'Sam Hauser': nan}

In [19]:
# convert dict to dataframe
Data = pd.DataFrame.from_dict(add, orient='index')

In [20]:
Data

,0
Aaron Nesmith,NaN
Al Horford,42.813931
Daniel Theis,NaN
Derrick White,11.567525
Grant Williams,22.470427
Jaylen Brown,56.285503
Jayson Tatum,43.846817
Juwan Morgan,NaN
Luke Kornet,NaN
Malik Fitts,NaN


In [21]:
# convert dataframe to csv
mypath = "Final_Score/LSTM"
if not os.path.isdir(mypath):
    os.makedirs(mypath)
Data.to_csv(f"{mypath}/BostonCeltics.csv", index=True)